In [57]:
from serpapi import GoogleSearch
import json
import os
from dotenv import load_dotenv
from anthropic import Anthropic


load_dotenv()

True

In [58]:

CLAUDE_API_KEY = os.environ.get("CLAUDE_API_KEY")
client = Anthropic(api_key=CLAUDE_API_KEY)

system_prompt = ""

try:
    with open("hotels_prompt1.txt", "r", encoding="utf-8") as f:
        system_prompt = f.read()
        
except FileNotFoundError:
    print("hotels_prompt1.txt not found")
    
else:
    def get_hotel_request(user_input):
        response = client.messages.create(
            model="claude-sonnet-4-20250514",   
            max_tokens=500,
            system=system_prompt,
            messages=[
                {"role": "user", "content": user_input}
            ]
        )
        raw = response.content[0].text.strip()
        return raw



In [78]:
import ast


user_text = "Book me a hotel in Dubai for 2 adults from Jan 10/2026 to Jan 15/2026, in price range of 250 to 400 SAR per night, with free cancellation and and pool and Free Wi-Fi."
params = get_hotel_request(user_text)

out_file = "hotel_params.json"

# convert params string to dict
if isinstance(params, str):
    s = params.strip()
    if s.startswith("params"):
        s = s.split("=", 1)[1].strip()
    try:
        params_dict = ast.literal_eval(s)
    except Exception:
        params_dict = json.loads(s)
else:
    params_dict = params

# write dict to json file
with open(out_file, "w", encoding="utf-8") as f:
    json.dump(params_dict, f, ensure_ascii=False, indent=2)
print("Hotel parameters written to hotel_params.json")


Hotel parameters written to hotel_params.json


In [79]:
import ast
SERPAPI_KEY = os.environ.get("SERPAPI_KEY")

# If get_hotel_request returned the dict as a string like "params = {...}", convert it to a dict
if isinstance(params, str):
    s = params.strip()
    if s.startswith("params"):
        s = s.split("=", 1)[1].strip()
    params = ast.literal_eval(s)

print("Search Parameters:", params)
params["api_key"] = SERPAPI_KEY


Search Parameters: {'engine': 'google_hotels', 'q': 'Hotels in Dubai', 'check_in_date': '2026-01-10', 'check_out_date': '2026-01-15', 'adults': '2', 'children': '0', 'currency': 'SAR', 'gl': 'sa', 'hl': 'en', 'min_price': '250', 'max_price': '400', 'free_cancellation': 'true', 'amenities': ['6', '35'], 'api_key': ''}


In [80]:
search = GoogleSearch(params)
results = search.get_dict()

with open("hotels.json", "w") as f:
    json.dump(results, f, indent=2)

In [81]:
clean_hotels = []
for hotel in results.get("properties", []):
    clean_hotels.append({
        "id": hotel.get("property_token"),
        "name": hotel.get("name"),
        "price": hotel.get("rate_per_night", {}).get("extracted_lowest"),
        "stars": hotel.get("extracted_hotel_class"),
        "rating": hotel.get("overall_rating"),
        "reviews": hotel.get("reviews"),
        "location_rating": hotel.get("location_rating"),
        "coordinates": hotel.get("gps_coordinates"),
        "amenities": hotel.get("amenities", []),
        "link": hotel.get("link"),
        "sponsored": hotel.get("sponsored")
    })


out_file = "hotels_filtered.json"
with open(out_file, "w", encoding="utf-8") as f:
    json.dump(clean_hotels, f, ensure_ascii=False, indent=2)

print(f"Wrote {len(clean_hotels)} hotels to {out_file}")


Wrote 18 hotels to hotels_filtered.json
